In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir "/content/drive/My Drive/final_dataset/labels"

Mounted at /content/drive
mkdir: cannot create directory ‘/content/drive/My Drive/final_dataset/labels’: File exists


# Dataset

In [4]:
imgs = os.listdir('/content/drive/My Drive/final_dataset/images')
imgs_train, imgs_val = train_test_split(imgs, test_size=0.05)

df = pd.read_csv('/content/drive/My Drive/final_dataset/annotation.csv')

In [5]:
import yaml
cwd = '/content/drive/My Drive/final_dataset/'
data = dict(
    train =  cwd + 'train.txt',
    val   =  cwd + 'val.txt',
    nc    = 1,
    names = ['licence'],
)

with open(cwd + 'bgr.yaml', 'w') as outfile:
    yaml.dump(data, outfile, default_flow_style=False)
with open(cwd + 'train.txt', 'w') as f:
    for path in imgs_train:
        f.write(cwd+'images/'+path+'\n')
with open(cwd + 'val.txt', 'w') as f:
    for path in imgs_val:
        f.write(cwd+'images/'+path+'\n')

In [6]:
for file in imgs:
    file = file.split('.')[0]
    bboxs = []
    for _,row in df[df['file'] == file].iterrows():
        bbox = [str(0), str(row['Xcent']), str(row['Ycent']), str(row['boxW']), str(row['boxH'])]
        bbox = ' '.join(bbox)
        bboxs.append(bbox)
    with open(cwd+'labels/'+file+'.txt', 'w') as f:
        bboxs = '\n'.join(bboxs)
        f.write(bboxs)

# YOLOv5

In [7]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 13715, done.
remote: Total 13715 (delta 0), reused 0 (delta 0), pack-reused 13715
Receiving objects: 100% (13715/13715), 13.23 MiB | 30.73 MiB/s, done.
Resolving deltas: 100% (9451/9451), done.
/content/yolov5
     |████████████████████████████████| 1.6 MB 37.9 MB/s 


In [8]:
!pip install -q wandb --upgrade

import wandb
wandb.login(anonymous='must')

     |████████████████████████████████| 1.9 MB 32.0 MB/s 
     |████████████████████████████████| 162 kB 44.7 MB/s 
     |████████████████████████████████| 182 kB 67.3 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 162 kB 65.4 MB/s 
     |████████████████████████████████| 158 kB 58.9 MB/s 
     |████████████████████████████████| 157 kB 49.1 MB/s 
     |████████████████████████████████| 157 kB 41.9 MB/s 
     |████████████████████████████████| 157 kB 68.1 MB/s 
     |████████████████████████████████| 157 kB 69.3 MB/s 
     |████████████████████████████████| 157 kB 64.8 MB/s 
     |████████████████████████████████| 157 kB 53.9 MB/s 
     |████████████████████████████████| 157 kB 66.4 MB/s 
     |████████████████████████████████| 156 kB 70.1 MB/s 


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Train

In [9]:
!python train.py --img 448\
--batch 16\
--epochs 65\
--data '/content/drive/My Drive/final_dataset/bgr.yaml'\
--weights yolov5m6.pt

wandb: Currently logged in as: anony-moose-415836. Use `wandb login --relogin` to force relogin
train: weights=yolov5m6.pt, cfg=, data=/content/drive/My Drive/final_dataset/bgr.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=65, batch_size=16, imgsz=400, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.2-188-g209be93 Python-3.7.14 torch-1.12.1+cu113 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr